### Basic Data Fetch Prototypes

Thie first one I found on the blog I recently published an article on:
https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859

Feel free to use the SDK API token I setup referenced in the `noaa` class.

In [164]:
# In case you don't have folium
# !pip install folium

In [165]:
import requests, pandas as pd, numpy as np, json
import folium
#parse the datetimes we get from NOAA
from datetime import datetime



#Long Beach Airport station
station_id = 'GHCND:USW00023129'

In [131]:
class noaa:
    
    base_url   = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate=2015-01-01&enddate=2015-12-31"
    noaa_token = 'gqfRhLiqfjECEAjlenVDCVImuElAuPGA'
    
    def __init__(self, **opts):
        for attr, value in opts.items():
            if hasattr(self, attr):
                setattr(self, attr, value)
        
    def request_json(self, url, **params):
        headers = {'token': self.noaa_token}
        result = requests.get(url, headers = headers)
        return result.json()
    
    def get_station(self, datasetid = "GHCND", datatypeid = "TAVG", stationid = "GHCND:USW00023129", startdate = "2015-01-01", enddate = "2015-12-31"):
        return self.request_json(self.base_url)
        
w = noaa()
response = w.get_station()

In [132]:
df = pd.DataFrame(response['results'])
df['date'] = pd.to_datetime(df['date'])

In [133]:
df

,date,datatype,station,attributes,value
0,2015-01-01,TAVG,GHCND:USW00023129,"H,,S,",81
1,2015-01-02,TAVG,GHCND:USW00023129,"H,,S,",89
2,2015-01-03,TAVG,GHCND:USW00023129,"H,,S,",99
3,2015-01-04,TAVG,GHCND:USW00023129,"H,,S,",112
4,2015-01-05,TAVG,GHCND:USW00023129,"H,,S,",148
...,...,...,...,...,...
360,2015-12-27,TAVG,GHCND:USW00023129,"H,,S,",98
361,2015-12-28,TAVG,GHCND:USW00023129,"H,,S,",92
362,2015-12-29,TAVG,GHCND:USW00023129,"H,,S,",99
363,2015-12-30,TAVG,GHCND:USW00023129,"H,,S,",104


### From the noaa-sdk

Someone on github published a package in teh pypi universe system.  Here's an example of its usage (uncomment the next line to install the package).

https://pypi.org/project/noaa-sdk/

In [39]:
# !pip install noaa_sdk

     |████████████████████████████████| 204kB 11.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 7.6MB/s  eta 0:00:01
  Created wheel for noaa-sdk: filename=noaa_sdk-0.1.17-cp37-none-any.whl size=12241 sha256=0541e51b0b5455b0a2eaab3f089903cca9d20f676698fb77a1a3fa0b9dac9bde
  Stored in directory: /Users/davidyerrington/Library/Caches/pip/wheels/23/b6/1d/6445bdc059d7d3653c654f05fab68b1017c644020a3f65eec2
  Created wheel for httplib2: filename=httplib2-0.10.3-cp37-none-any.whl size=83988 sha256=28162f14614b2ad65dead4ff77ab7e9671e37b4bbcd05bb544a7c9623bfb99e1
  Stored in directory: /Users/davidyerrington/Library/Caches/pip/wheels/d6/f4/9c/f4eab4c19c0bde393b00a1f83afe12cc469852ff3810cd6f6d
Successfully built noaa-sdk httplib2
  Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0


### Lookup Weather by Zipcode

The return format is a generator which is a type not compatible with DataFrame inputs so we will convert each obervation to a list for quick view.  I do believe we could create an easy mapping of this data that could automatically translate to a nice convention like `feature_name_unit_type`.

> The service seems to work great, except for when you plug seldovia zipcode in. Will try to lookup by station next.

In [215]:
from noaa_sdk import noaa
n = noaa.NOAA()

## List of stations

In [151]:
station_results = n.stations()

In [175]:
stations = []
## normalized structure
for row in station_results['features']:
    stations.append({
        'station_name': row['properties']['name'],
        'uri':          row['id'],
        'id':           row['properties']['stationIdentifier'],
        'lat':          row['geometry']['coordinates'][1],
        'long':         row['geometry']['coordinates'][0],
        'elevation_m':  row['properties']['elevation']['value'],
    })
    
df_stations = pd.DataFrame(stations)
df_stations.head(10)

,station_name,uri,id,lat,long,elevation_m
0,"Phoenix, Phoenix Sky Harbor International Airport",https://api.weather.gov/stations/KPHX,KPHX,33.427799,-112.003465,339.8520
1,"San Francisco, San Francisco International Air...",https://api.weather.gov/stations/KSFO,KSFO,37.619610,-122.365580,3.0480
2,Boise Air Terminal,https://api.weather.gov/stations/KBOI,KBOI,43.567040,-116.240530,860.1456
3,"Dodge City, Dodge City Regional Airport",https://api.weather.gov/stations/KDDC,KDDC,37.772780,-99.969720,790.0416
4,"Salt Lake City, Salt Lake City International A...",https://api.weather.gov/stations/KSLC,KSLC,40.770690,-111.965030,1288.0848
5,"Denver, Denver International Airport",https://api.weather.gov/stations/KDEN,KDEN,39.846580,-104.656220,1647.1392
6,"Great Falls, Great Falls International Airport",https://api.weather.gov/stations/KGTF,KGTF,47.473330,-111.382220,1120.1400
7,Rogue Valley International Airport,https://api.weather.gov/stations/KMFR,KMFR,42.375030,-122.876960,399.8976
8,"Albuquerque, Albuquerque International Airport",https://api.weather.gov/stations/KABQ,KABQ,35.041670,-106.614720,1630.9848
9,"Los Angeles, Los Angeles International Airport",https://api.weather.gov/stations/KLAX,KLAX,33.938060,-118.388890,38.1000


## Stations of Interest

In [185]:
name_mask = df_stations['station_name'].str.contains("Homer|Seldovia|Kenai|Nome|Adak|Fairbanks")
df_stations[name_mask]

,station_name,uri,id,lat,long,elevation_m
294,"Fairbanks, Fairbanks International Airport",https://api.weather.gov/stations/PAFA,PAFA,64.80389,-147.87611,131.9784
312,"Nome, Nome Airport",https://api.weather.gov/stations/PAOM,PAOM,64.51194,-165.44500,10.9728
586,"Kenai, Kenai Municipal Airport",https://api.weather.gov/stations/PAEN,PAEN,60.57306,-151.24500,29.8704
638,"Homer, Homer Airport",https://api.weather.gov/stations/PAHO,PAHO,59.65000,-151.48333,25.9080
743,"Fairbanks, Eielson Air Force Base",https://api.weather.gov/stations/PAEI,PAEI,64.65000,-147.10000,167.0304
1000,St. Johnsbury - Fairbanks,https://api.weather.gov/stations/K1V4,K1V4,44.42010,-72.01930,212.1408
1161,"Adak Island, Adak Airport",https://api.weather.gov/stations/PADK,PADK,51.87778,-176.64583,6.0960
1590,"Seldovia, Seldovia Airport",https://api.weather.gov/stations/PASO,PASO,59.45000,-151.70000,9.1440
2061,Homerville Airport,https://api.weather.gov/stations/KHOE,KHOE,31.05590,-82.77410,56.3880


### Basic Map Exploration of Stations

> Warning: Loading all US stations into map.. will chunk CPU in your browser

In [214]:
station_match = "Seldovia" # carefull of matching more than one at once when updating this value
lat, long = df_stations[df_stations['station_name'].\
                       str.contains("Seldovia")][['lat','long']].values[0]
f_map = folium.Map(
    location   = (lat, long),
    tiles      = 'Stamen Terrain',
    zoom_start = 5
)
## Add markers to map from DataFrame
def apply_markers(row):
    tooltip_meta = f"<div style='width: 300px;'><strong>{row['station_name']}</strong></br>"
    tooltip_meta += f"id: {row['id']}</br> elivation/m: {row['elevation_m']}</div>"
    folium.Marker([row['lat'], row['long']], popup=tooltip_meta).add_to(f_map)
    
df_stations.apply(apply_markers, axis = 1)  

## Display map
f_map

## Updated: Geting observations by station
Rather than zipcode.  Seems that the web api is inconsistent to get the data from NOAA by zipcode we we will be using the station by id.

### Beginning of our own API
This class will be designed to perform the following tasks:
- Fetch data by station ID (known as "observations")
- Auto-normalize / flatten data with units
- Save data as files
- View raw data from response to see what the noaa service has sent us (for testing)

Things to be done:
- Integrate Hig's code
- Discuss any naming conventions
- Setup a script to automate this stuff to a SQLite instance (really easy to manage for now)
  - This is when we have it run from a list of station ID's
  - Automatically fetch historical data (need to do more research.  Easy to do -- just time.)
- Test the code maybe setup basic unit tests
- Host these datasets somewhere else other than our laptops eventually

In [ ]:
class noaa_requests:
    
    station_data_result = []
    station_data        = []
    noaa_service        = False 
    
    def __init__(self, **params):
        for attr, value in params.items():
            if(hasattr(self, attr)):
                setattr(self, attr, value)
                
    def set_station_data(self, station_id = "PASO"):
        self.station_data_result = [] # reset data
        self.station_data_result = self.noaa_service.stations_observations(station_id)
        
    def normalize_station_data(self):
        self.station_data = [] # reset result whenever called
        for obs in self.station_data_result['features'][0:10]:
            meta_observation = {}
            for key, item in obs['properties'].items():
                if type(item) == dict and item.get('unitCode'):
                    meta_name = f"{key}_{item['unitCode'].split(':')[1]}"
                    meta_observation[meta_name] = item['value']
                elif type(item) == list:
                    for item_key, item_value in item[0].items():
                        unit_id = item_value.get('unitCode', False) if type(item_value) == dict else False
                        if unit_id:
                            item_key += "_" + unit_id
                            item_value = item_value.get("value")
                        multi_item_key = f"{key}_{item_key}"
                        meta_observation[multi_item_key] = item_value
                else:
                    meta_observation[key] = item
            self.station_data.append(meta_observation)
            
    def to_df(self):
        return pd.DataFrame(self.station_data)

# This library actually makes the request
n  = noaa.NOAA()

# the noaa_sdk is passed to our class and used internally throughout
nr = noaa_requests(noaa_service = n)
nr.set_station_data("PASO")
nr.normalize_station_data()

## FYI: Looking at the raw results from NOAA web service

I stored the semi-raw request in an internal class attribute/variable called `self.station_data_result`.

> In the event we're _certain_ an item should exist in our _cleaned_ dataset that our dataframes are showing, we can always inspect this object for sanity and/or QA purposes.

Viewing the first 2 results from NOAA before normalization.

In [288]:
nr.station_data_result['features'][0:2]

[{'id': 'https://api.weather.gov/stations/PASO/observations/2020-02-22T22:53:00+00:00',
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-151.69, 59.44]},
  'properties': {'@id': 'https://api.weather.gov/stations/PASO/observations/2020-02-22T22:53:00+00:00',
   '@type': 'wx:ObservationStation',
   'elevation': {'value': 9, 'unitCode': 'unit:m'},
   'station': 'https://api.weather.gov/stations/PASO',
   'timestamp': '2020-02-22T22:53:00+00:00',
   'rawMessage': 'PASO 222253Z AUTO 16006KT 1SM -SN VV016 M08/M11 A2973 RMK AO2 SLP069 P0000 T10781111 TSNO',
   'textDescription': 'Light Snow',
   'icon': 'https://api.weather.gov/icons/land/day/snow?size=medium',
   'presentWeather': [{'intensity': 'light',
     'modifier': None,
     'weather': 'snow',
     'rawString': '-SN'}],
   'temperature': {'value': -7.7999999999999545,
    'unitCode': 'unit:degC',
    'qualityControl': 'qc:V'},
   'dewpoint': {'value': -11.099999999999966,
    'unitCode': 'unit:degC',
    'qualityC

## Getting Station Data as DataFrame

Once you've initialized the NOAA class, and given it a handle to the noaa_sdk instance like so:

```python
n  = noaa.NOAA()
nr = noaa_requests(noaa_service = n)
nr.set_station_data("PASO") # makes request
nr.normalize_station_data() # normalizes/flattens structure with unit labels
```

You can use the following method to get a clean _DataFrame_.

> We already have an instance of "nr" so just reuse it.


In [296]:
df_paso = nr.to_df()
df_paso.head(10)

,@id,@type,elevation_m,station,timestamp,rawMessage,textDescription,icon,presentWeather_intensity,presentWeather_modifier,...,maxTemperatureLast24Hours_degC,minTemperatureLast24Hours_degC,precipitationLastHour_m,precipitationLast3Hours_m,precipitationLast6Hours_m,relativeHumidity_percent,windChill_degC,heatIndex_degC,cloudLayers_base_unit:m,cloudLayers_amount
0,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T22:53:00+00:00,PASO 222253Z AUTO 16006KT 1SM -SN VV016 M08/M1...,Light Snow,https://api.weather.gov/icons/land/day/snow?si...,light,None,...,None,None,0,NaN,None,77.176123,-12.981111,None,490,VV
1,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T22:50:00+00:00,PASO 222250Z AUTO 17006KT 1SM -SN VV015 M08/M1...,Light Snow,https://api.weather.gov/icons/land/day/snow?si...,light,None,...,None,None,0,NaN,None,79.007598,-13.221965,None,460,VV
2,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T22:15:00+00:00,PASO 222215Z AUTO 1/2SM SN VV014 M08/M11 A2972...,Snow,https://api.weather.gov/icons/land/day/snow?si...,None,None,...,None,None,0,NaN,None,80.228310,NaN,None,430,VV
3,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T22:08:00+00:00,PASO 222208Z AUTO 16007KT 130V190 1/2SM SN VV0...,Snow,https://api.weather.gov/icons/land/day/snow?si...,None,None,...,None,None,0,NaN,None,80.228310,NaN,None,460,VV
4,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T21:53:00+00:00,PASO 222153Z AUTO VRB06G16KT 1SM -SN OVC017 M0...,Light Snow,https://api.weather.gov/icons/land/day/snow?si...,light,None,...,None,None,0,NaN,None,77.176123,-12.981111,None,520,OVC
5,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T21:42:00+00:00,PASO 222142Z AUTO 18008G16KT 150V220 1SM -SN O...,Light Snow,https://api.weather.gov/icons/land/day/snow?si...,light,None,...,None,None,0,NaN,None,77.176123,NaN,None,520,OVC
6,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T21:21:00+00:00,PASO 222121Z AUTO VRB05G16KT 3/4SM -SN OVC017 ...,Light Snow,https://api.weather.gov/icons/land/day/snow?si...,light,None,...,None,None,0,NaN,None,80.303848,-12.391140,None,520,OVC
7,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T21:07:00+00:00,PASO 222107Z AUTO 17011G16KT 140V210 1SM -SN O...,Light Snow,https://api.weather.gov/icons/land/day/snow?si...,light,None,...,None,None,0,NaN,None,80.303848,NaN,None,520,OVC
8,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T20:53:00+00:00,PASO 222053Z AUTO 18008G16KT 140V230 3/4SM -SN...,Light Snow,https://api.weather.gov/icons/land/day/snow?si...,light,None,...,None,None,0,0.0,None,80.303848,NaN,None,520,OVC
9,https://api.weather.gov/stations/PASO/observat...,wx:ObservationStation,9,https://api.weather.gov/stations/PASO,2020-02-22T20:33:00+00:00,PASO 222033Z AUTO 19010G18KT 1/2SM SN VV012 M0...,Snow,https://api.weather.gov/icons/land/day/snow?si...,None,None,...,None,None,0,NaN,None,80.303848,-14.743837,None,370,VV


# Everything from here out is older code.  Next update I will clean up this notebook more.  Thanks - Dave

> The class I just wrote today (2/22/2020 cool date!) replaces all this stuff here and will work with any station by ID rather than zipcode.

#### Observations to DataFrame

Much of the observations are still encoded as dictionaries (not a bad thing).  I can write an apply function to encode these as new features while saving the old ones.  If we like this data, we can deal with it before we put it in DataFrames.

In [65]:
observations = n.get_observations('11365','US') # 99663 = seldovia, but it seems service doesn't respond
data = []
for row in observations:
    data.append(row)
    
df_noaa = pd.DataFrame(data)
df_noaa['timestamp'] = pd.to_datetime(df_noaa['timestamp'])

In [135]:
df_noaa.columns.tolist()

['@id',
 '@type',
 'elevation',
 'station',
 'timestamp',
 'rawMessage',
 'textDescription',
 'icon',
 'presentWeather',
 'temperature',
 'dewpoint',
 'windDirection',
 'windSpeed',
 'windGust',
 'barometricPressure',
 'seaLevelPressure',
 'visibility',
 'maxTemperatureLast24Hours',
 'minTemperatureLast24Hours',
 'precipitationLastHour',
 'precipitationLast3Hours',
 'precipitationLast6Hours',
 'relativeHumidity',
 'windChill',
 'heatIndex',
 'cloudLayers']

In [67]:
df_noaa.head()

,@id,@type,elevation,station,timestamp,rawMessage,textDescription,icon,presentWeather,temperature,...,visibility,maxTemperatureLast24Hours,minTemperatureLast24Hours,precipitationLastHour,precipitationLast3Hours,precipitationLast6Hours,relativeHumidity,windChill,heatIndex,cloudLayers
0,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 01:51:00+00:00,KLGA 120151Z 30013KT 10SM FEW021 BKN100 OVC110...,Cloudy,https://api.weather.gov/icons/land/night/ovc?s...,[],"{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 70.6580455433216, 'unitCode': 'unit:...","{'value': 4.208512575318366, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 640, 'unitCode': 'unit:m'}..."
1,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 01:31:00+00:00,KLGA 120131Z 31014KT 10SM SCT021 OVC110 08/04 ...,Cloudy,https://api.weather.gov/icons/land/night/ovc?s...,[],"{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 76.3607448618594, 'unitCode': 'unit:...","{'value': 4.049072026674992, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 640, 'unitCode': 'unit:m'}..."
2,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 00:58:00+00:00,KLGA 120058Z 31009KT 10SM -RA FEW019 BKN026 OV...,Light Rain,https://api.weather.gov/icons/land/night/rain?...,"[{'intensity': 'light', 'modifier': None, 'wea...","{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 5.0123538466526725, 'unitCode': 'uni...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 580, 'unitCode': 'unit:m'}..."
3,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-12 00:51:00+00:00,KLGA 120051Z 31011KT 10SM SCT022 BKN041 OVC050...,Cloudy,https://api.weather.gov/icons/land/night/ovc?s...,[],"{'value': 7.800000000000011, 'unitCode': 'unit...",...,"{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 4.55996919337781, 'unitCode': 'unit:...","{'value': None, 'unitCode': 'unit:degC', 'qual...","[{'base': {'value': 670, 'unitCode': 'unit:m'}..."
4,https://api.weather.gov/stations/KLGA/observat...,wx:ObservationStation,"{'value': 9, 'unitCode': 'unit:m'}",https://api.weather.gov/stations/KLGA,2020-02-11 23:51:00+00:00,KLGA 112351Z 30005KT 10SM OVC060 08/05 A2982

### Encoding to new features is actually not that bad

This function will translate the lower level format as new columns on select columns.  I've labeld them below as `to_encode`.  In this example, we use an `apply` function on our data which allows you to run the given function on every column (axis = 0 (default)), or rows (axis = 1).  This is preferable to basic Python iteration because Pandas optimizes these operations against its own unique "special" data types under the hood.  

I have a few tutorials that I wrote on this type of feature within Pandas and I can upload it somewhere if you let me know.

> Protip:  When in Pandas, use `.map` and `.apply` on your DataFrames to make changes rather than a for loop or other iterators.

In [126]:
to_encode = ['elevation', 'temperature', 'visibility', 'minTemperatureLast24Hours', 'precipitationLastHour', 'precipitationLast3Hours', 'precipitationLast6Hours', 'relativeHumidity', 'windChill', 'heatIndex', 'cloudLayers']

def encode_value_to_unit(row):
    for column in row.index:
        try:
            row[f"{column}_{row[column]['unitCode']}"] = row[column]['value']
        except:
            print(f"Couldn't encode feature for {column}")
    return row

    
df_noaa_encoded = df_noaa[to_encode].apply(encode_value_to_unit, axis = 1)

Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers
Couldn't encode feature for cloudLayers


### Still cruft in the DataFrame

New features were added to the end and they look pretty clean.

In [128]:
df_noaa_encoded.head()

,elevation,temperature,visibility,minTemperatureLast24Hours,precipitationLastHour,precipitationLast3Hours,precipitationLast6Hours,relativeHumidity,windChill,heatIndex,...,elevation_unit:m,temperature_unit:degC,visibility_unit:m,minTemperatureLast24Hours_unit:degC,precipitationLastHour_unit:m,precipitationLast3Hours_unit:m,precipitationLast6Hours_unit:m,relativeHumidity_unit:percent,windChill_unit:degC,heatIndex_unit:degC
0,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 70.6580455433216, 'unitCode': 'unit:...","{'value': 4.208512575318366, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,0.0,NaN,NaN,70.658046,4.208513,NaN
1,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 76.3607448618594, 'unitCode': 'unit:...","{'value': 4.049072026674992, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,0.0,NaN,NaN,76.360745,4.049072,NaN
2,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': 0, 'unitCode': 'unit:m', 'qualityCon...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 5.0123538466526725, 'unitCode': 'uni...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,0.0,NaN,NaN,79.084822,5.012354,NaN
3,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 7.800000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 79.08482210318559, 'unitCode': 'unit...","{'value': 4.55996919337781, 'unitCode': 'unit:...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,7.8,16090,None,NaN,NaN,NaN,79.084822,4.559969,NaN
4,"{'value': 9, 'unitCode': 'unit:m'}","{'value': 8.300000000000011, 'unitCode': 'unit...","{'value': 16090, 'unitCode': 'unit:m', 'qualit...","{'value': None, 'unitCode': 'unit:degC', 'qual...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': None, 'unitCode': 'unit:m', 'quality...","{'value': 0.0005, 'unitCode': 'unit:m', 'quali...","{'value': 79.71241189231577, 'unitCode': 'unit...","{'value': 6.737267429777944, 'unitCode': 'unit...","{'value': None, 'unitCode': 'unit:degC', 'qual...",...,9,8.3,16090,None,NaN,NaN,0.0005,79.712412,6.737267,NaN


### Save encoded data to file for review

In [129]:
df_noaa.to_csv("../data/processed/noaa_from_api.csv", encoding = "UTF8")